## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-30-05-52-08 +0000,nypost,"Nearly 37,000 VA employees furloughed or worki...",https://nypost.com/2025/10/30/us-news/departme...
1,2025-10-30-05-47-49 +0000,nypost,Texas maintenance workers poisoned by fentanyl...,https://nypost.com/2025/10/30/us-news/texas-ma...
2,2025-10-30-05-43-00 +0000,wsj,Trump Says U.S. Will Begin Testing Nuclear Wea...,https://www.wsj.com/politics/national-security...
3,2025-10-30-05-38-00 +0000,wsj,"Trump Meets With Xi, Declares Immediate Cut to...",https://www.wsj.com/world/china/trump-meets-xi...
4,2025-10-30-05-37-20 +0000,nyt,In Photos: Trump’s Asia Trip,https://www.nytimes.com/2025/10/28/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2370/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,45
134,new,28
20,will,18
36,hurricane,15
51,melissa,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
34,2025-10-30-02-28-18 +0000,nypost,Trump says ‘we have a deal’ with China’s Xi Ji...,https://nypost.com/2025/10/29/us-news/trump-ch...,127
29,2025-10-30-03-06-01 +0000,wapo,Trump directs Pentagon to test nuclear weapons...,https://www.washingtonpost.com/politics/2025/1...,108
8,2025-10-30-05-25-09 +0000,wapo,Trump cuts tariffs on China after ‘12 out of 1...,https://www.washingtonpost.com/politics/2025/1...,105
38,2025-10-30-02-08-41 +0000,nypost,Trump orders Pentagon to ‘immediately’ restart...,https://nypost.com/2025/10/29/us-news/trump-or...,101
28,2025-10-30-03-22-48 +0000,wapo,Trump says South Korea will be able to build n...,https://www.washingtonpost.com/world/2025/10/2...,101


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
34,127,2025-10-30-02-28-18 +0000,nypost,Trump says ‘we have a deal’ with China’s Xi Ji...,https://nypost.com/2025/10/29/us-news/trump-ch...
100,70,2025-10-29-22-57-52 +0000,nypost,Jack Ciattarelli says NJ will welcome any New ...,https://nypost.com/2025/10/29/us-news/jack-cia...
49,68,2025-10-30-01-32-00 +0000,wsj,Federal Reserve Chair Jerome Powell delivered ...,https://www.wsj.com/economy/central-banking/fe...
170,60,2025-10-29-20-56-14 +0000,wapo,Hurricane Melissa hits Cuba; has killed at lea...,https://www.washingtonpost.com/weather/2025/10...
239,52,2025-10-29-18-09-32 +0000,nypost,Fed cuts interest rates by quarter point for s...,https://nypost.com/2025/10/29/business/fed-cut...
244,50,2025-10-29-18-05-17 +0000,wapo,Report: Government shutdown will cost the econ...,https://www.washingtonpost.com/business/2025/1...
299,49,2025-10-29-15-08-02 +0000,nypost,Nvidia becomes first $5T company powered by AI...,https://nypost.com/2025/10/29/business/nvidia-...
284,46,2025-10-29-16-04-58 +0000,bbc,"Israeli strikes in Gaza kill 104, health minis...",https://www.bbc.com/news/articles/cgjdy5eevn2o...
33,43,2025-10-30-02-40-50 +0000,latimes,He said he was an immigration lawyer but raped...,https://www.latimes.com/california/story/2025-...
262,39,2025-10-29-17-19-00 +0000,wsj,"Paramount Lays Off Thousands of Workers, Inclu...",https://www.wsj.com/business/media/paramount-l...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
